# Data Preparation

In [1]:
#importing the libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
sb.set()

In [2]:
#loading raw data
sneakerData = pd.read_excel('StockX-Data-Contest-2019-3.xlsx',sheet_name=1) #sheet_name=1 for sneaker data
sneakerData.info()

#cleaning the dates column
sneakerData['Order Date'] = sneakerData['Order Date'].astype('datetime64[ns]')
sneakerData['Release Date'] = sneakerData['Release Date'].astype('datetime64[ns]')
sneakerData['Turnover Days'] = sneakerData['Order Date'] - sneakerData['Release Date']

#cleaning the price column
sneakerData['Profit'] = sneakerData['Sale Price'] - sneakerData['Retail Price']
sneakerData['Profit Ratio'] = (sneakerData['Profit'] / sneakerData['Retail Price']).round(2)

#breaking the Sneaker Name into Model Variants (categorical data)
sneakerData['Sneaker Name'] = sneakerData['Sneaker Name'].str.lower()

sneakerData["Model"] = sneakerData['Sneaker Name'].apply(
    lambda x : 'yeezy-boost-350' if 'yeezy' in x.split("-") else (
        'air-jordan-1-retro-high' if 'jordan' in x.split('-') else (
            'air-force-1' if 'force' in x.split('-') else(
                'air-max-90' if '90' in x.split('-') else (
                    'air-max-97' if '97' in x.split('-') else (
                        'air-presto' if 'presto' in x.split('-') else (
                            'air-vapormax' if 'vapormax' in x.split('-') else (
                                'blazer-mid' if 'blazer' in x.split('-') else (
                                    'react-hyperdunk-2017-flyknit' if 'hyperdunk' in x.split('-') else (
                                        'zoom-fly' if 'zoom' in x.split('-') else (np.nan)
                                    )
                                )
                            )
                        )
                    )
                )
            )
        )
    )
)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99956 entries, 0 to 99955
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Order Date    99956 non-null  datetime64[ns]
 1   Brand         99956 non-null  object        
 2   Sneaker Name  99956 non-null  object        
 3   Sale Price    99956 non-null  float64       
 4   Retail Price  99956 non-null  int64         
 5   Release Date  99956 non-null  datetime64[ns]
 6   Shoe Size     99956 non-null  float64       
 7   Buyer Region  99956 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(3)
memory usage: 6.1+ MB


In [3]:
#check if all the Sneaker Names are categorised into Model Variants
uncategorised_model = pd.DataFrame()
uncategorised_model[sneakerData['Model'].isnull() == True]
print(uncategorised_model)

Empty DataFrame
Columns: []
Index: []


<ipython-input-3-3e9c9a106138>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  uncategorised_model[sneakerData['Model'].isnull() == True]


uncategorised_model is an empty DF, hence all sneakers are categorised into Model Variants

In [5]:
#importing the colour information from stockX website, based on each sneaker name
colourway = pd.read_excel('supplemental_data_colorway.xlsx')
colourway['Style'] = colourway['Style'].str.lower()
colourway.head()

#merging the colourway DF with the sneakerData DF
sneakerData = pd.merge(sneakerData,colourway,how='left',left_on='Sneaker Name',right_on='Style')

In [6]:
#Dropping columns that are irrelevant to the analysis
parsed_data = sneakerData.drop(['Sale Price','Retail Price','Style','Number of Sales','Website','Product Line'],axis=1)
parsed_data['Turnover Days'] = parsed_data['Turnover Days'].dt.days
parsed_data['Turnover Weeks'] = (parsed_data['Turnover Days'] / 7).round(0)
parsed_data = parsed_data.drop(parsed_data[parsed_data['Turnover Weeks'] < 0].index)
parsed_data = parsed_data.drop(parsed_data[parsed_data['Turnover Weeks'] > 52].index)

#consolidating the primary colors into 1 single column
def get_col(row):
    for color in parsed_data.columns[10:20]:
        if row[color] == 1:
            return color
parsed_data['color'] = parsed_data.apply(get_col, axis=1)
parsed_data.to_csv('parsed_data.csv')